### BW40 - super로 부모 클래스를 초기화하자.

- 자식 클래스에서 부모 클래스를 초기화하는 오래된 방법은
<br/> 바로 자식 인스턴스에서 부모 클래스의 __ init__ 메서드를 직접 호출하는 것이다.

In [1]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

- 기본적인 클래스 계층의 경우 작동함, 다중 상속에 의해 영향을 받으면 예측할 수 없는 방식으로 작동한다.
<br/> → 문제 중 하나는 모든 하위 클래스에서 __ init__ 호출의 순서가 정해져 있지 않다는 것이다.

In [11]:
class TimesTwo: # 부모 클래스를 각각 정의
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

In [3]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print('첫 번째 부모 클래스 순서에 따른 값은 (5 * 2) + 5 =', foo.value)

첫 번째 부모 클래스 순서에 따른 값은 (5 * 2) + 5 = 15


- 부모 클래스의 순서에 따라 초기화가 실행되는 결과를 볼 수 있다.
<br/> Ex. 다음은 부모 클래스 순서를 다르게 나열한 경우

In [4]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

bar = AnotherWay(5)
print('두 번째 부모 클래스 순서에 따른 값은', foo.value)

두 번째 부모 클래스 순서에 따른 값은 15


- 부모 클래스의 생성자 순서는 그래로라서 똑같은 결과가 나온다.
<br/> 즉, 클래스 정의에서 부모클래스를 나열한 순서에 따르지 않는다. ( 일치놉 )
<br/><br/> 호출과 나열 순서가 달라서 생기는 문제는 발견하기가 쉽지가 않다.

<hr/>

Ex. 다이아몬드 상속으로 인해 다른 문제가 생길 수도 있다.
> 다이아몬드 상속 ?
어떤 클래스가 두 가지 서로 다른 클래스를 상속하는데, 
<br/> 거슬러 올라가면 같은 조상 클래스가 존재하는 것

- 다이아몬드 상속은 공통 조상의 __ init__ 메서드를 여러번 호출 될 수 있기 때문이다.

In [12]:
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

In [13]:
class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

In [14]:
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)

In [15]:
foo = ThisWay(5)
print('(5 * 7) + 9 = 44가 나와야 하지만 실제로는', foo.value)


(5 * 7) + 9 = 44가 나와야 하지만 실제로는 14


- 두 번째 부모 클래스의 생성자 PlusNine.__ init__을 호출하면
<br/> MyBaseClas.__ init__이 다시 호출되면서 self.value가 5로 돌아간다.
<br/> 따라서 self.value는 5 + 9 = 14가 된다.
<br/> → super라는 내장 함수와 표준 메서드 결정 순서가 있다.
<br/><br/> 다이아몬드 계층의 공통 상위 클래스를 단 한번만 호출하도록 보장한다.
<br/> 그리고 MRO 즉, 표준메서드는 상위 클래스를 초기화하는 순서를 정의한다, 이때 C3 선형화라는 알고리즘 사용

In [ ]:
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7

In [ ]:
class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

- 다이아몬드의 정점에 있는 부모 클래스는 한번만 실행되고, 생성자는 class 문에 지정된 순서대로 호출 된다.

In [ ]:
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

In [6]:
foo = GoodWay(5)
print('7 * (5 + 9) = 98이 나와야 하고 실제로도', foo.value)

7 * (5 + 9) = 98이 나와야 하고 실제로도 98


- 7 * 5 + 9로 생각할 수 있지만 생성자 순서는 MRO 정의에 따른다

In [8]:
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


- 역순으로 생성자가 호출된다.
- super().__ init__ 호출은 다중 상속을 튼튼하게 해주며, 
<br/> 하위 클래스에서 MyBaseClass.__ init__을 직접 호출하는 것보다 유지보수를 더 편하게 해준다.
<br/> 나중에 클래스 이름을 바꿔도 super로 호출할 수 있다.
<br/><br/> super 함수에 두 가지 파라미터를 넘길 수 있다,
<br/> 첫번째 파라미터는 내가 접근하고 싶은 MRO 뷰를 제공할 부모 타입 두번째는 MRO 뷰에 접근할 떄 사용할 인스턴스다.

In [9]:
class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

- 하지만 object 인스턴스를 초기화할 때는 두 파라미터를 지정할 필요가 없다.
- 클래스 정의 안에서 아무 인자도 지정하지 않고 super를 호출하면, 
<br/> 파이썬 컴파일러가 자동으로 올바른 파라미터(__ class__와 self)를 넣어준다.

In [16]:
class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3

class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value /= 3

assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

- super에 파라미터를 제공해야 하는 유일한 경우는 자식 클래스에서 상위 클래스의 특정 기능에 접근해야하는 경우뿐이다.

### 기억하자
- 파이썬은 표준 메서드 결정 순서(MRO)를 활용해 상위 클래스 초기화 순서와 다이아몬드 상속 문제를 해결한다.
- 부모 클래스를 초기화할 때는 super 내장 함수를 아무 인자 없이 호출하자.
<br/> super를 아무 인자 없이 호출하면 파이썬 컴파일러가 자동으로 올바른 파라미터를 넣어준다.